Import Required Libraries and Spark SQL Functions

In [0]:
import os
from pyspark.sql.functions import *

Define Gold and Silver Layer Base Paths

In [0]:
silver_base = "/Volumes/adventure_works_lakehouse/adventure_works/lakehouse/silver"
gold_base   = "/Volumes/adventure_works_lakehouse/adventure_works/lakehouse/gold"

Read Silver Returns Data

In [0]:
df_returns = spark.read.format("delta").load(f"{silver_base}/silver_adventureworks_returns")

Transform Returns Data for Fact Table

In [0]:
df_fact_returns = (
    df_returns
    .withColumn("DateKey", date_format(col("ReturnDate"), "yyyyMMdd").cast("int"))
    .select(
        "DateKey",
        "ProductKey",
        "TerritoryKey",
        "ReturnQuantity"
    )
)

In [0]:
df_fact_returns.limit(7).display()

DateKey,ProductKey,TerritoryKey,ReturnQuantity
20150118,312,9,1
20150118,310,10,1
20150121,346,8,1
20150122,311,4,1
20150202,312,6,1
20150215,312,1,1
20150219,311,9,1


Write Returns Fact Table to Gold Layer

In [0]:
df_fact_returns.write.format("delta")\
        .mode("overwrite")\
        .save(f"{gold_base}/fact_returns")